In [19]:
import pandas as pd
import json
import os
import folium
import requests
import time

def visualize_routes_v2():
    print("🗺️  Memulai Visualisasi Peta (Versi Detail)...")
    
    # 1. SETUP PATH
    BASE_DIR = os.getcwd()
    if os.path.basename(BASE_DIR) == 'notebook':
        BASE_DIR = os.path.dirname(BASE_DIR)
    DATA_DIR = os.path.join(BASE_DIR, 'dataset')
    RES_DIR = os.path.join(BASE_DIR, 'results')
    FIG_DIR = os.path.join(BASE_DIR, 'figures')
    os.makedirs(FIG_DIR, exist_ok=True)
    
    CSV_FILE = os.path.join(DATA_DIR, 'final_dataset_2echelon.csv')
    ROUTES_FILE = os.path.join(RES_DIR, 'optimized_routes.json')
    
    if not os.path.exists(ROUTES_FILE):
        print("❌ File hasil rute tidak ditemukan. Jalankan script modelling dulu.")
        return

    # 2. LOAD DATA
    df = pd.read_csv(CSV_FILE)
    nodes_map = df.set_index('Node_ID').to_dict('index')
    
    with open(ROUTES_FILE, 'r', encoding='utf-8') as f:
        routes_data = json.load(f)
        
    # 3. SETUP PETA (Bandung)
    m = folium.Map(location=[-6.9175, 107.6191], zoom_start=13, tiles='CartoDB positron')
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']
    
    # 4. PLOT SEMUA NODE (TITIK LOKASI)
    print("📍 Plotting titik lokasi...")
    for nid, info in nodes_map.items():
        loc = [info['Lat'], info['Lon']]
        
        # Konten Popup HTML yang rapi
        popup_html = f"""
        <div style="width:200px">
            <b>{info['Nama Fasilitas']}</b><br>
            <hr style="margin:5px 0">
            Type: {info['Type']}<br>
            Demand: {info.get('Demand_Ton', 0)} Ton<br>
            Biaya Gerobak: Rp {info.get('Cost_Gerobak', 0):,}
        </div>
        """
        
        if info['Type'] == 'TPS':
            folium.CircleMarker(
                location=loc, radius=4, color='#333', weight=1, fill=True, fill_color='white', fill_opacity=1,
                popup=folium.Popup(popup_html, max_width=250),
                tooltip=info['Nama Fasilitas']
            ).add_to(m)
        elif info['Type'] == 'POOL':
            folium.Marker(
                location=loc, icon=folium.Icon(color='blue', icon='warehouse', prefix='fa'),
                popup=folium.Popup(popup_html, max_width=250),
                tooltip=f"POOL: {info['Nama Fasilitas']}"
            ).add_to(m)
        elif info['Type'] == 'TPA':
            folium.Marker(
                location=loc, icon=folium.Icon(color='red', icon='trash', prefix='fa'),
                popup=folium.Popup(popup_html, max_width=250),
                tooltip="TPA SARIMUKTI"
            ).add_to(m)

    # 5. GAMBAR RUTE (JALAN ASLI)
    def get_geometry(waypoints):
        # OSRM Request
        coords = ";".join([f"{lon},{lat}" for lat, lon in waypoints])
        url = f"http://router.project-osrm.org/route/v1/driving/{coords}?overview=full&geometries=geojson"
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                geo = r.json()['routes'][0]['geometry']['coordinates']
                return [[y, x] for x, y in geo]
        except:
            pass
        return waypoints

    print(f"🖌️  Menggambar {len(routes_data)} rute truk...")
    
    for i, truck in enumerate(routes_data):
        node_ids = truck['Route_Nodes']
        
        # Ambil koordinat
        waypoints = []
        for nid in node_ids:
            if nid in nodes_map:
                waypoints.append([nodes_map[nid]['Lat'], nodes_map[nid]['Lon']])
        
        if len(waypoints) < 2: continue
        
        # Request OSRM
        path = get_geometry(waypoints)
        color = colors[truck['Kecamatan_ID'] % len(colors)]
        
        # Info Tooltip untuk Garis Rute
        tooltip_text = f"""
        Vehicle: {truck['Vehicle']}
        Load: {truck['Total_Load']} Ton
        Dist: {truck['Total_Dist']} km
        Ritase: {truck['Rit_Count']} x
        """
        
        folium.PolyLine(
            path, color=color, weight=3, opacity=0.8,
            tooltip=tooltip_text.strip()
        ).add_to(m)
        
        time.sleep(0.1) # Rate limit

    out_map = os.path.join(FIG_DIR, 'peta_optimasi_rute.html')
    m.save(out_map)
    print(f"✅ Peta Visualisasi selesai! Buka file: {out_map}")

if __name__ == "__main__":
    visualize_routes_v2()

🗺️  Memulai Visualisasi Peta (Versi Detail)...
📍 Plotting titik lokasi...
🖌️  Menggambar 151 rute truk...
✅ Peta Visualisasi selesai! Buka file: d:\Arya\Kuliah\Semester 7\TRO\Tubes\figures\peta_final_multitrip.html
